In [1]:
%config Completer.use_jedi = False
import sys
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import time
import threading
import gc
import copy

sys.path.append("/end/home/rh2310/morpho_repo")
from turing.utils import *
import turing.pinns as tu
import turing.pinns_multi_nodes as tu_multi
from turing.loss_functions import *

GPU = False
import os

if GPU:
    txt_device = 'gpu:0'
else:
    txt_device = 'cpu:0'    
    os.environ["CUDA_VISIBLE_DEVICES"]="-1"

2022-07-08 13:31:57.602960: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1892] Ignoring visible gpu device (device: 0, name: Quadro K2000, pci bus id: 0000:05:00.0, compute capability: 3.0) with Cuda compute capability 3.0. The minimum required Cuda capability is 3.5.


# Load data

In [2]:
data_path = os.path.abspath("turing.npy")
with open(data_path, 'rb') as f:
    data = np.load(f)
    
data_path = os.path.abspath("turing_t.npy")
with open(data_path, 'rb') as f:
    t_star = np.load(f) 
    
T = t_star.shape[0]    
    
L = 50
x_size = data.shape[1]
y_size = data.shape[2]
N = x_size*y_size

model_params = {'training_data_size': T*N,#T*32,
                'pde_data_size': (T*N)//(32),
                'boundary_data_size':((x_size + y_size)*T)//(8)}

dataset = create_dataset(data, t_star, N, T, L, **model_params)
lb = dataset['lb']
ub = dataset['ub']
obs_X = dataset['obs_input']
obs_Y = dataset['obs_output']
pde_X = dataset['pde']


# Test

In [3]:
layers = [3, 64, 64, 64, 64, 2]

pinn = tu.NN(layers, lb, ub, dtype=tf.float64)
pde_loss = Schnakenberg(dtype=tf.float64, D_u = 0.005, D_v = 0.2)
#pde_loss = Schnakenberg_multi(dtype=tf.float64, D_u = 0.005, D_v = 0.2)
#non_zero_loss = Non_zero_params(pde_loss.name, [pde_loss.D_u, pde_loss.D_v])
#periodic = Periodic_boundary(dataset['boundary_LB'], dataset['boundary_RT'], pinn.dtype)

In [4]:
model = tu.TINN(pinn, #tu_multi.TINN_multi_nodes(pinn, 
                                  pde_loss,
                                  extra_loss = [], 
                                 # nodes_n = 2,
                                  #node_names= ["a", "s"],
                                 # alpha = 0.25
               )

In [5]:
results = model.train(epochs = 2,
                      batch_size = 512,
                      X = obs_X,
                      Y = obs_Y,
                      print_interval=1,
                      stop_threshold = 1e-5,
                      shuffle = True,
                      sample_losses = True,              
                      sample_regularisations = True,
                      sample_gradients = True)


Start of epoch 0
Training observations acc over epoch: 1.07756
total loss: 5246.40736, total regularisd loss: 5246.40736
obs u loss: 1134.70912, obs v loss: 1072.13355
pde u loss: 1100.99508, pde v loss: 1938.56962
lambda obs u: 1.00000, lambda obs v: 1.00000
lambda pde u: 1.00000, lambda pde v: 1.00000
c_0: 10.33260 c_1: 9.87785 c_2: 9.91118 c_3: 9.95110 

Time taken: 49.86s

Start of epoch 1
Training observations acc over epoch: 0.96262
total loss: 2002.42347, total regularisd loss: 2002.42347
obs u loss: 967.58294, obs v loss: 1003.87235
pde u loss: 14.57061, pde v loss: 16.39757
lambda obs u: 1.17913, lambda obs v: 1.82070
lambda pde u: 0.50007, lambda pde v: 0.50009
c_0: 10.87768 c_1: 9.67343 c_2: 9.79514 c_3: 9.81069 

Time taken: 32.60s


In [8]:
type(len(obs_X))

int